# Basic scraping for Guardian discussions u

In [1]:
! ./.g


[master 159945f] web
 12 files changed, 2720 insertions(+), 498 deletions(-)
 rename .ipynb_checkpoints/{Untitled-checkpoint.ipynb => quad_forms-checkpoint.ipynb} (100%)
 create mode 100644 .ipynb_checkpoints/random_shapes-checkpoint.ipynb
 create mode 100644 None0000000.png
 create mode 100644 ejdxt.pkl
 create mode 100644 eyhx8.pkl
 create mode 100644 eytkx.pkl
 create mode 100644 femicides3.csv
 rewrite liberation_femicide.ipynb (78%)
 create mode 100644 links.txt
 rename Untitled.ipynb => quad_forms.ipynb (100%)
 create mode 100644 random_shapes.ipynb
Enumerating objects: 15, done.
Counting objects: 100% (15/15), done.
Delta compression using up to 4 threads
Compressing objects: 100% (11/11), done.
Writing objects: 100% (11/11), 1.43 MiB | 93.00 KiB/s, done.
Total 11 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 2 local objects.
To https://github.com/macbuse/GUARDIAN.git
   07063a7..159945f  master -> master


In [59]:
with open('guardian comments scraper.ipynb','r') as fp:
    nb = fp.read()

In [60]:
import re

pp = re.compile("(https://www.the.*?)'")
lks = pp.findall(nb)

with open('links.txt','a') as fp:
    fp.write('\n'.join(lks))

In [2]:
import re 
import requests
import pickle
import json

In [14]:
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/19/ditch-the-algorithm-generation-students-a-levels-politics'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/21/joe-biden-democrat-convention-nice-guy-trump-fight'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/23/the-cruel-exams-algorithm-has-laid-bare-the-unfairness-at-the-heart-of-our-schools'
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/01/us-mainstream-right-vigilante-terror'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/31/politicians-covid-19-statistics-statisticians'

In [132]:
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/29/rival-scientists-lockdowns-scientific-covid-19'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/08/johnson-and-sturgeon-will-need-a-better-philosophy-than-precaution-to-rid-us-of-covid'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/10/continual-local-lockdowns-answer-covid-control'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/11/the-rebel-scientists-cause-would-be-more-persuasive-if-it-werent-so-half-baked'

In [21]:
key = my_keys.keys['guardian']

NameError: name 'my_keys' is not defined

In [146]:
#scrape the page and get the short url
short_url_pp =  re.compile('"shortUrlId":"(.*?)"')

r = requests.get(article_url)
mm = short_url_pp.search(r.text)
short_url = mm.group(1)

#the short url allows us to access the discussion
comments_url = 'https://api.nextgen.guardianapps.co.uk/discussion/%s.json'%short_url
params = {'orderBy' : 'newest',
          'pageSize' : 25,
          'displayThreaded' : 'true',
          'commentsClosed' : 'false',
          'page' : '',
          'maxResponses' : 50
         }

json_data =  []
start_page  = 1
last_page = None


for num_page in range(start_page, 10**4):
    print('getting comments page no.', num_page)
    params['page'] = num_page
    r = requests.get( comments_url, params=params)
    json_data.append( r.text)
    #The last page is a field in the json object
    #I don't need to read this as json I could do a regexp
    if num_page == start_page:
        json_dict = json.loads(r.text)
        last_page = json_dict['lastPage']
        print('there are %d pages'%last_page)
    if num_page == last_page : break
        

fn = '%s.pkl'%short_url.split('/')[-1]
print('archiving to %s'%fn)
with open(fn ,'wb') as fp:
    pickle.dump(json_data,fp)
    


getting comments page no. 1
there are 8 pages
getting comments page no. 2
getting comments page no. 3
getting comments page no. 4
getting comments page no. 5
getting comments page no. 6
getting comments page no. 7
getting comments page no. 8
archiving to f4a3q.pkl


## remember each page of comments has been stored separately

In [134]:
dd = json.loads(json_data[0])
len(json_data)

8

In [135]:
xx = json.loads(json_data[0])
xx.keys()

dict_keys(['postedCommentHtml', 'commentsHtml', 'commentCount', 'refreshStatus', 'lastPage', 'paginationHtml'])

In [136]:
print( xx['commentsHtml'][200:700])

 class="d-comment
            d-comment--top-level"
           id="comment-144422422"
           data-comment-id="144422422"
           data-comment-author="Hushing"
           data-comment-replies="0"
           data-comment-timestamp="2020-10-11T14:25:06.000+01:00"
           data-comment-author-id="12407494"
           data-comment-highlighted="false"
           itemscope itemtype="http://schema.org/Comment">

    <div class="d-comment__inner d-comment__inner--top-level">
        <div class="


In [92]:
pa = re.compile('data-comment-author="(.*?)"')
pid = re.compile('data-comment-author-id="(\d+)')
pcid = re.compile('\sid="comment-(.*?)"')
pp = re.compile('<li class="d-comment(.*?)>',re.DOTALL)

auths = []
md = []
for xx in json_data:
    hh = json.loads(xx)['commentsHtml']
    auths.extend(list( zip(pa.findall(hh), 
                           pid.findall(hh)
                          )))
    md.extend(pp.findall(hh))

auths = list(set(auths))
auths.sort(key=lambda x : x[0].lower())


In [102]:
#print( '\n'.join([x for x in md if 'pinkie' in x.lower()]))

https://discussion.theguardian.com/comment-permalink/144375739

In [34]:
auths = dict(auths)

In [54]:
comments_url

'https://api.nextgen.guardianapps.co.uk/discussion//p/f3ke9.json'

In [147]:
class Comment():
    pass

from bs4 import BeautifulSoup

def html2comments(x):
    tt = json.loads(x)
    soup = BeautifulSoup(tt['commentsHtml'],  "lxml")
    raw = [ comment.text for comment in  
            soup.findAll('div', {'class' : "d-comment__body"}) ]
    
    metadata = soup.findAll('span', {'class' : "d-comment__author"})
    
    
    auth = [ x.text.upper().strip() for x in metadata]
    auth_id = [ x.find('a')['href'] for x in metadata]
    #this was a pain to do really
    #the problem is blocked comments
    comment_id = [ x['id'][len('comment-'):] for x in soup.findAll('li')]
 
    
    ss = '##{}\nhttps://discussion.theguardian.com/comment-permalink/{}\n{}\n\n{}'
    return [ ss.format(a,b,d,c.strip())  
                   for a,b,d,c in zip(auth, comment_id, auth_id, raw)]
     
all_texts = [ html2comments(page) for page in json_data[:] ]



from itertools import chain
all_comments = list(chain.from_iterable(all_texts))
    

# Random data hacking

In [148]:
A = [x.split('\n')[0] for x in all_comments]
from collections import Counter
AN = Counter(A)
sorted([(x,y) for x,y in AN.items() if y > 4], key=lambda x : -x[1]) 

[('##ROCKYREX', 25),
 ('##DAVE6723', 22),
 ('##23X7', 20),
 ('##MREVOICE', 19),
 ('##SHARLIII', 18),
 ('##OLDWEASE', 17),
 ('##WOOPSGROINK', 15),
 ('##PETEKREFF', 14),
 ('##GEORGEHOWARD', 13),
 ('##HARIF_', 12),
 ('##DDTQM', 12),
 ('##WHERESMEHAT', 11),
 ('##STEVE__M', 11),
 ('##BICBIRO', 10),
 ('##ELEUTHERA_GIBBONS', 10),
 ('##TAGGARTT', 10),
 ('##JESSTHECRIP', 10),
 ('##BIELSA', 9),
 ('##WHATENLIGHTENMEANT', 9),
 ('##BELLAS1', 9),
 ('##DUBGUARD', 9),
 ('##SOCALALEX', 8),
 ('##KARLPRESSBURG', 8),
 ('##ENVIROCAPITALIST', 8),
 ('##CHRISFF2', 8),
 ('##TREBELL', 7),
 ('##DOESNOTEXIST', 7),
 ('##ARTISTFRMLYKNWNAS', 7),
 ('##HFLASHMAN', 7),
 ('##HAK_A_DALAN', 7),
 ('##JULESBYWATERLEES', 6),
 ('##TENIENTESNAFU', 6),
 ('##SEELIFEDIFFERENTLY', 6),
 ('##MINTAKA', 6),
 ('##STRONTIUM90', 6),
 ('##RATHERBEINOZ', 6),
 ('##NATHANSPRING', 6),
 ('##PINKIE123', 6),
 ('##DELTAJONES', 6),
 ('##ISHOWERDAILY', 6),
 ('##ALIENINLONDON', 6),
 ('##MINCEPI', 6),
 ('##SPOTTHELEMON', 5),
 ('##BRAMBALUS', 5),
 ('#

In [149]:
! ./.g


[master b8c8a75] web
 2 files changed, 1457 insertions(+), 1317 deletions(-)
 rewrite guardian comments scraper.ipynb (76%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 47.51 KiB | 84.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/macbuse/GUARDIAN.git
   d207214..b8c8a75  master -> master


In [150]:
print(all_comments[0])

##HUSHING
https://discussion.theguardian.com/comment-permalink/144422422
https://profile.theguardian.com/user/id/12407494

Any proposal should be clear in its conception, its aims, and what outcomes it would take the person proposing to reconsider.
What facts are you drawing on?What facts are you dismissing?What preconceptions are you making?What assumptions are you relying on?What do you aim to achieve? Why?How does what you propose achieve this?What do you aim to avoid? Why?How does what you propose avoid this?What answers to serious (non-strawman) criticisms do you have?What would you consider to be a sign that your proposal had failed?
Anything that doesn't answer these questions is performative bullshit.


In [151]:
all_comments.sort(key=lambda x : -len(x) )
print( '\n\n'.join(all_comments[:20]) )

##PETEKREFF
https://discussion.theguardian.com/comment-permalink/144419786
https://profile.theguardian.com/user/id/3809933

The result was “the Great Barrington declaration”, which prompted headlines about the disintegrating scientific consensus that managing Covid-19 requires society-wide changes to behaviour. It is a sorry parable about what happens when bad science gets co-opted by shady ideological interests.

I thought we were meant to leave Covid-19 strategy to the medical experts and not to *checks Wikipedia* PPE graduates. 

giving it the sheen of academic respectability

For some reason you fail to mention the authors' names (until mentioning Professor Gupta a long way down the article) so readers could check them out and see what qualifications they have. 

The professors do not define who is “vulnerable”

They don't have to. Everyone who's been following the news recently knows that the elderly and people with co-morbidities are vulnerable. 

The declaration sets itself up a

In [18]:
#https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gregmcshane/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
a = '''My views haven't changed. Leaving the EU will be of no benefit to anyone except those who hold money and power along with certain corporates and a smidgen of medium to small businesses.
How else can it be when you are shutting shop to the most developed nations, the richest nations in the world, who are either in the EU or have market agreements with the EU. Even the emerging markets of China and India along with other developing economies are aspiring to develop trading agreements with the EU, the second largest economy in the world. Any deal with the UK will be secondary and will depend upon their agreement with the EU.
Boris Johnson once said "f**k business". I take this as meaning there is something in it for them and they don't really care if businesses find it difficulA
t to cope.
I can see ministers and conservative MPs queuing to fill their pockets by acting as advisors for potential trade deals whilst the rest of the country are left struggling trying to rebuild their lives, especially following the pandemic. The same ministers will probably be looking to lower all kinds of standards from food and welfare to safety at work to improve their opportunities.
When it comes to brinkmanship, I can only view this as getting away with whatever they can. I do not believe it is anything to do with negotiating an agreement that will be to the benefit of the UK or the EU as a whole, I don't believe our ministers are capable of doing that.'''

In [24]:
sid.polarity_scores(a)
sid.sentiment_valence(a)

TypeError: sentiment_valence() missing 4 required positional arguments: 'sentitext', 'item', 'i', and 'sentiments'

In [152]:
target = 'decolon'
target = 'science'
target = 'science'


xx = [x for x in all_comments if target.lower() in x.lower() and  len(x) > 20]
yy = [x for x in all_comments if '#pin' in x.lower() and 2000 > len(x) > 20]

#print('\n\n'.join( ['\n\n'.join(x) for x in zip(xx,yy)])  )


#print( '\n\n'.join(xx).replace(target, ) )
pxx = re.compile(target, re.I)
print( pxx.sub('**' + target.upper() + '**', 
              '\n\n'.join(xx)) )
               
#print( '\n\n'.join(xx) )


##PETEKREFF
https://discussion.theguardian.com/comment-permalink/144419786
https://profile.theguardian.com/user/id/3809933

The result was “the Great Barrington declaration”, which prompted headlines about the disintegrating scientific consensus that managing Covid-19 requires society-wide changes to behaviour. It is a sorry parable about what happens when bad **SCIENCE** gets co-opted by shady ideological interests.

I thought we were meant to leave Covid-19 strategy to the medical experts and not to *checks Wikipedia* PPE graduates. 

giving it the sheen of academic respectability

For some reason you fail to mention the authors' names (until mentioning Professor Gupta a long way down the article) so readers could check them out and see what qualifications they have. 

The professors do not define who is “vulnerable”

They don't have to. Everyone who's been following the news recently knows that the elderly and people with co-morbidities are vulnerable. 

The declaration sets itself 

36